# Code to Predict Earthquakes in California

This code was used to produce the results in section 4.3. The steps are as follows:

1. Create a grid of equally spaced (10km) points across the region of interest (California)

2. For each point investigate a set of final magnitudes and end dates

    (a) Magnitudes: 6.50, 6.75, 7.00 and 7.25, corresponding to radii: 138 km, 170 km, 209 km and 257 km according to the formula of Jaume and Sykes [#Jaume:1999]

    (b) Dates: end of each year from 1995 to 2011

3. Collect the data for all earthquakes within the radius and before the end date

4. Fit a straight-line and a power-law to the cumulative Benioff strain from the data, using the selected magnitude to calculate the final cumulative Benioff strain and finding the other parameters (including t_{f}, from the fit)

5. Mark the grid point as “positive” if 

    (a) t_{f} is greater than the date of the last data point but less than the date of the last data point plus 5 years

    (b) The variance from a power-law is less than half the variance from a straight-line fit i.e. the curvature parameter is <0.5

    (c) There are at least 10 data points

6. Create clusters of 10 or more positive points if the distance between them are always less than 100 km. I carried out this clustering using agglomerative hierarchical clustering in Python

7. Find the average t_{f} for each cluster


In [ ]:
#Import required packages
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt, pi
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import shapely.geometry
import pyproj
%matplotlib inline

import os

from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import math

from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch

## Prepare the Data

In [ ]:
#Read in the data - ANSS earthquake catalogue
data_path = "Data/ANSS_catalogue.csv"df = pd.read_csv(data_path, encoding = 'latin')
print(df.shape)

#Convert the date to datetime and check the date range
df.Date = pd.to_datetime(df.Date, format = '%d/%m/%Y')
print(df.Date.min())
print(df.Date.max())

#Add columns for the year, years since 1900 and days since 1900 (rounded to the nearest year)
df['year'] = df['Date'].dt.year
df['year_1900'] = df['year'] - 1900
df['day_1900'] = df['year_1900']*365

df.head(2)

In [ ]:
#Sort the dataset by magnitude
df.sort_values(['Magnitude'], ascending = False, inplace = True)

In [ ]:
'''
Remove aftershocks (rough method)
If more than one earthquake occurs on the same day
in the same approximate location keep only the largest earthquake
'''
df['lat_round'] = round(df.Latitude,0)
df['long_round'] = round(df.Longitude,0)
df.drop_duplicates(['Date','lat_round','long_round'],inplace = True)
df.drop(['lat_round','long_round'], axis = 1, inplace = True)
print(df.shape)


In [ ]:
#Calculate the energy release in Nm
df['Energy_log10'] = 4.8 + 1.5*df['Magnitude']
df['Energy'] = 10 ** df.Energy_log10
df.head(2)


In [ ]:
#Calculate the critical radius (from Jaume and Sykes 1999)
df['R_log10'] = -0.2 + 0.36*df['Magnitude']
df['R_km'] = 10 ** df.R_log10
df.head(2)

In [ ]:
#Drop some columns
df.drop(['Time','Depth','NbStations','Gap','Distance','RMS','Source','EventID'], axis = 1, inplace = True)

#Convert latlong from degrees to radians
df['lat_rad'] = (df.Latitude*pi)/180
df['lon_rad'] = (df.Longitude*pi)/180

## Define Helper Functions

In [ ]:
"""
A function to calculate the great circle distance between two points 
on the earth (specified in radians)
This function was taken from the internet
"""
def haversine(target_lon, target_lat, all_lon, all_lat): 
    dlon = all_lon - target_lon 
    dlat = all_lat - target_lat 
    # haversine formula
    a = np.sin(dlat/2)**2 + np.cos(target_lat) * np.cos(all_lat) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in km. Use 3956 for miles
    return c * r

In [ ]:
"""
A Function to return all earthquakes within a radius of a target point
and with magnitude greater than M_min
"""
def get_points(target_point, all_points, radius = 100, M_min = 4):
        
    #Filter to only earthquakes over the set magnitude
    all_points = all_points[all_points.Magnitude >= M_min]   
    
    #Filter to only records before the date of the target point (and the target point itself)
    all_points = all_points[all_points.Date <= target_point.Date]
    #print("Number of earlier earthquakes: %d"  % all_points.shape[0])
    
    #Calculate the distance between the target point and all other points
    all_points['distance'] = haversine(target_point.lon_rad,target_point.lat_rad,
                                       all_points.lon_rad, all_points.lat_rad)
    
    #Filter to only points within the radius
    all_points = all_points[all_points['distance'] <= radius]
    #print("Number of earlier earthquakes within %d km: %d"  % (radius, all_points.shape[0]))
    
    #Calculate the time delta in days
    all_points['Time_Delta'] = (target_point.Date - all_points.Date).dt.days
    
    #Return the filtered dataset of earthquakes
    return(all_points)


In [ ]:
"""
Function to order a table of earthquakes by date 
and calculate their cummulative Benioff strain
"""
def benioff_calc(all_points):
    #Sort the earthquakes by date
    all_points.sort_values('Date', inplace = True)
    
    #Calculate the cummulative Benioff strain
    all_points['Energy_sqrt'] = np.sqrt(all_points.Energy)
    all_points['Cummulative_Benioff_Strain'] = all_points['Energy_sqrt'].cumsum()
    
    #Return the original dataset with a new column for the cummulative Benioff strain
    return(all_points)

In [ ]:
"""
Function to calculate the rmse for predictions
"""
def rmse_calc(pred, all_points):
    #Calculate rmse
    err = all_points.Cummulative_Benioff_Strain - pred
    err_squared = err**2
    mean_err = np.mean(err_squared)
    rmse = np.sqrt(mean_err)
    #Return the rmse
    return(rmse)

## Define Fitting Functions

In [ ]:
"""
Function to fit a straight-line to data
"""
def straight_line_fit(all_points):
    #Define the straight line function and parameters
    def straightLine(t, A, B):
        return A + B*t
    
    #Fit the data
    popt, pcov = curve_fit(straightLine,
                           all_points.day_1900, #t
                           all_points.Cummulative_Benioff_Strain
                          )
    #Make the predictions and calculate the rmse
    pred = popt[0] + popt[1]*all_points.day_1900
    rmse = rmse_calc(pred, all_points)
    
    return(popt, rmse)


In [ ]:
"""
Function to fit a power-law to data
"""
def power_law_fit(all_points, final_benioff):
    #Define the power law function and the parameters
    def power_law(t, B, tf, m):
        return B*(tf - t)**m

    #Set up a starting point for the fitting parameters
    starting_point = [-1000000, 40000, 0.3]
    #Fit the data
    popt, pcov = curve_fit(power_law,
                           all_points.day_1900, #t
                           all_points.Cummulative_Benioff_Strain - final_benioff,
                           p0 = starting_point,
                           maxfev = 10000
                          )
    #Make the predictions and calculate the rmse 
    pred = final_benioff + popt[0]*(popt[1] - all_points.day_1900)**popt[2]
    rmse = rmse_calc(pred, all_points)
    
    return(popt, rmse)


## Fitting Master Function

In [ ]:
"""
Function to fit the data using the different methods (straight-line and power-law)
Returns the dataset used for the fit, the curvature parameter 
and the power law fitting parameters
"""
def fitting_function(target_point, df, r):
    #Get the relevant earthquakes within the radius
    sub_points = get_points(target_point, df, radius = r,M_min = target_point.Magnitude - 2)
    
    #Check to see if there are enough points to attempt to fit the data
    if(sub_points.shape[0] < 10):
        return

    #Calculate the cummulative benioff strain
    sub_points = benioff_calc(sub_points)

    #Set the index
    sub_points.index = sub_points.day_1900

    #Fit to a straight line
    sl_opt, sl_rmse = straight_line_fit(sub_points)

    #Try to fit to a power law
    final_benioff = max(sub_points.Cummulative_Benioff_Strain) + target_point.Energy_sqrt
    try:
        pl_opt, pl_rmse = power_law_fit(sub_points, final_benioff)
    except:
        return

    #Calculate C
    c = pl_rmse/sl_rmse
                
    return(sub_points, c, pl_opt)
    

## Grid Setup


In [ ]:
"""
Create an grid of points with 10km spacings across California
"""
#Projection setup
p_ll = pyproj.Proj('epsg:4326') #latlong
p_mt = pyproj.Proj('epsg:3310') #projected metric - for California

#Set the corners of the area to be split into a grid
#Set to slightly larger than the area covered by the earthquakes used in Bowman 1998
nw = shapely.geometry.Point((-123, 38))
se = shapely.geometry.Point((-114, 32))

stepsize = 10000 # 10 km grid step size

# Project corners to target projection
s = pyproj.transform(p_ll, p_mt, nw.y, nw.x)
e = pyproj.transform(p_ll, p_mt, se.y, se.x)
print(s)
print(e)

#Loop through all grid points to find the latlong values
grid_points_lat = []
grid_points_long = []
x = s[0]
while x < e[0]:
    y = s[1]
    while y > e[1]:
        #Create a geometry point with the required projection
        p = shapely.geometry.Point(pyproj.transform(p_mt, p_ll, x, y))
        grid_points_lat.append(p.x)
        grid_points_long.append(p.y)
        y -= stepsize
    x += stepsize    

#Setup the grid as a dataframe
grid = pd.DataFrame({'grid_id': range(0,len(grid_points_long)),'lat': grid_points_lat,'long': grid_points_long})
grid.to_csv('California_grid.csv', index = False)    

print(grid.shape)   
grid.head(2)


## Fit the Data for Each Grid Point

In [ ]:
"""
Set up the magnitudes, radii and final dates to loop through
Note that the radii corespond to the magnitudes (using the formula of Jaume and Sykes)
"""
magnitudes = [6.50, 6.75, 7.00, 7.25]
dates = pd.date_range(start='1/1/1995', end = '1/1/2012', freq='Y')
radii = [138, 170, 209, 257]
print(len(magnitudes)*len(dates)*grid.shape[0])

In [ ]:
'''
Loop through each grid point, critical region radius/final magnitude and end date
Collect the data for all earthquakes within the radius and before the end date
Fit a straight-line and a power-law on all parameters
'''
#Set up a dataframe to store the results
grid_results = pd.DataFrame(columns = ['Date', 'Lat','Long','Magnitude',
                                       'R', 'C', 'tf','B','m','data_points'])
for i in range(0, grid.shape[0]):
    print(i)
    lat = grid.lat.iloc[i]
    long = grid.long.iloc[i]
    
    for mag in magnitudes:
        for date in dates:
            r = 10**(-0.2 + 0.36*mag)
            #Setup a target point 
            target_point = pd.Series({'Date': date,
                                      'Latitude': lat,
                                      'Longitude': long,
                                     'lat_rad':lat*pi/180,
                                     'lon_rad': long*pi/180,
                                      'Magnitude': mag,
                                      'Energy_sqrt': np.sqrt(10 ** (4.8 + 1.5*mag))
                                     })
            
            #Fit the straight-line and the power-law
            result = fitting_function(target_point, df, r)
            #If it was not possible to fit a power-law -> move to the next grid point
            if(result == None):
                continue
            sub_points, c, pl_opt = result
            
            #Append the results to the dataframe
            fit_result = {'Date': date,
                          'Lat': lat,
                          'Long': long,
                          'Magnitude': mag,
                          'R': r,
                          'C': c,
                          'tf': pl_opt[1],
                          'B': pl_opt[0],
                          'm': pl_opt[2],
                          'data_points': sub_points.shape[0],
                         'final_date': max(sub_points.day_1900)}
            grid_results = grid_results.append(fit_result, ignore_index=True)
            
    #Save out after every grid point - necessary as the code takes a long time to run
    grid_results.to_csv('California_grid_analysed.csv', index = False)  


## Process the Results

In [ ]:
"""
Housekeeping - required as I saved out multiple files from the previous step
"""
#Read in all files and combine
mypath = os.getcwdb()
files = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
files = [f.decode('utf-8') for f in files]
files = [f for f in files if "California_grid_analysed" in f]

all_files = []
for f in files:
    all_files.append(pd.read_csv(f))
    
grid_results = pd.concat(all_files)
print(grid_results.shape)

#Remove duplicate values
grid_results['Check'] = grid_results.Date.map(str) + grid_results.Lat.map(str) + \
                        grid_results.Long.map(str) + grid_results.Magnitude.map(str)
grid_results = grid_results.drop_duplicates(['Check'])
print(grid_results.shape)
grid_results.drop(['Check'], axis = 1, inplace = True)

In [ ]:
"""
Mark each row as “positive” if:
    (a) t_f is > the date of the last data point but < the date of the last data point plus 5 years
    (b) The variance from the power law is less than half the variance from a straight line fit
    (c) There are at least 10 data points
"""
grid_results['Date_Check'] = (grid_results.tf > grid_results.final_date) & \
                                (grid_results.tf < grid_results.final_date + 5*365)
print(grid_results.Date_Check.value_counts())
grid_results['C_Check'] = (grid_results.C < 0.5)
print(grid_results.C_Check.value_counts())
grid_results['Points_Check'] = (grid_results.data_points >=10)
print(grid_results.Points_Check.value_counts())
grid_results['Positive'] = grid_results.Date_Check & grid_results.C_Check & grid_results.Points_Check
print(grid_results.Positive.value_counts())


In [ ]:
#Filter to just positive grid points
positive_results = grid_results[grid_results.Positive].copy()
positive_results.to_csv('Positive_California_Grid_Points.csv', index = False)
grid_results.head(2)

In [ ]:
"""
Create clusters of 10 or more positive points 
if the distance between them are always less than 100km
use agglomerative hierarchical clustering
"""
#Convert the location to radians
positive_results['lat_rad'] = (positive_results.Lat*pi)/180
positive_results['lon_rad'] = (positive_results.Long*pi)/180

#Loop through each year and radius/magnitude
for year in positive_results.Date.unique():
    for r in positive_results.R.unique():
        print("Year: %s, R: %.2f" % (year, r))

        #Filter to only results from the year and radius/magnitude
        temp_df = positive_results[(positive_results.Date == year) &\
                                   (positive_results.R == r)].copy()
        
        #If there are < 5 positive grid points move to the next item in the loop
        if(temp_df.shape[0] < 5):
            continue
           
        #Compute the distances between all points
        X = temp_df.iloc[:,[1,2]].values
        distances = cdist(X,X, lambda ori,des: int(round(haversine(ori[1]*pi/180, ori[0]*pi/180,
                                                           des[1]*pi/180, des[0]*pi/180))))
        
        #Perform hierarchical aglomerative clustering on the distances
        #Using a distance threshold of 100km
        model = AgglomerativeClustering(n_clusters=None, affinity='precomputed',
                                linkage='single', distance_threshold = 100)
        model.fit(distances)
        labels = model.labels_
        print(model.n_clusters_)
        
        #Add the labels back into the dataframe
        temp_df['Cluster'] = labels
        temp_df.head()
        
        #For each cluster - calculate the tf (mean, standard deviation and count)
        #and the lat and long means i.e. the cluster centre
        cluster_details = temp_df.groupby(['Cluster'], as_index = False).\
                            agg({'tf':['mean', 'std', 'count'], 
                                 'Lat':'mean',
                                 'Long':'mean'})
        cluster_details.columns = ['Cluster','Cluster_tf_mean',
                                   'Cluster_tf_stdev', 'Cluster_Count',
                                   'lat_mean', 'long_mean']
        #Merge with the main dataset
        temp_df = temp_df.merge(cluster_details, how = 'left', on = 'Cluster')
        
        #Set clusters with <10 points to None
        temp_df.Cluster = np.where(temp_df.Cluster_Count < 10, None, temp_df.Cluster)
        
        #Convert the mean tf to a datetime
        temp_df['year'] = temp_df['Cluster_tf_mean'].apply(lambda x: str(x/365 + 1900).split('.')[0])
        temp_df['day'] = temp_df['Cluster_tf_mean'].apply(lambda x: '0.' + str(x/365 + 1900).split('.')[1])
        temp_df['day'] = temp_df['day'].apply(lambda x: int(float(x)*365) + 1)
        temp_df['date_tf'] = temp_df['day'].astype(str) + '/' + temp_df['year'].astype(str)
        temp_df['date_tf'] = pd.to_datetime(temp_df['date_tf'], format = '%j/%Y')
        
        temp_df.drop(['Date_Check', 'C_Check', 'Points_Check',
                      'lat_rad', 'lon_rad', 'year','day'],
                     axis = 1, inplace = True)

        #Save out
        temp_df.to_csv('Images/Clusters/' + year + '_' + str(round(r,2)) + '_Clusters.csv', index = False)
        
        